In [1]:
from Nystromformer.LRA.code import lra_config
from Nystromformer.LRA.code.dataset import LRADataset
#from Nystromformer.LRA.code.run_tasks import training_config
from torch.utils.data import DataLoader, RandomSampler
from torch import nn
import torch.nn.functional as F

piaynTaskDataDir = "Nystromformer/LRA/datasets"
piaynTaskModelDir = "Nystromformer/LRA"
task = "retrieval"

#get training config
training_config = lra_config.config[task]["training"]

#Check Train Config
print('Training Config: ', training_config)

#get pre-defined model config
model_config = lra_config.config[task]['model']

#Check model Config
print('Model Config: ', model_config)

#Get the dataset
train_dataset = LRADataset(piaynTaskDataDir + f"/{task}/{task}.train.pickle", False)
val_dataset = LRADataset(piaynTaskDataDir + f"/{task}/{task}.dev.pickle", False)
test_dataset = LRADataset(piaynTaskDataDir + f"/{task}/{task}.test.pickle", False)

#Create DataLoader iterators
ds_iter = {
    "train":enumerate(DataLoader(train_dataset, 
                                 #Sample batches randomly for number of specified steps
                                 sampler = RandomSampler(train_dataset, 
                                                         replacement=True, 
                                                         num_samples= training_config["num_train_steps"]*lra_config.config[task]['dataset']['train']), 
                                 batch_size = training_config["batch_size"], 
                                 drop_last = True)),
    "dev":enumerate(DataLoader(val_dataset, batch_size = 1, drop_last = True)),
    "test":enumerate(DataLoader(test_dataset, batch_size = 1, drop_last = True)),
}



Training Config:  {'batch_size': 32, 'learning_rate': 0.0001, 'warmup': 800, 'lr_decay': 'linear', 'weight_decay': 0, 'eval_frequency': 300, 'num_train_steps': 30000, 'num_eval_steps': 565}
Model Config:  {'learn_pos_emb': True, 'tied_weights': False, 'embedding_dim': 64, 'transformer_dim': 64, 'transformer_hidden_dim': 128, 'head_dim': 32, 'num_head': 2, 'num_layers': 2, 'vocab_size': 512, 'max_seq_len': 4000, 'dropout_prob': 0.1, 'attention_dropout': 0.1, 'pooling_mode': 'MEAN', 'num_classes': 2}
Loaded Nystromformer/LRA/datasets/retrieval/retrieval.train.pickle... size=147086
Loaded Nystromformer/LRA/datasets/retrieval/retrieval.dev.pickle... size=18090
Loaded Nystromformer/LRA/datasets/retrieval/retrieval.test.pickle... size=17437


In [2]:
#Check sizes of batches
batch = next((ds_iter['train']))
for k,v in batch[1].items():
  print(k,v.shape)

input_ids_0 torch.Size([32, 4096])
mask_0 torch.Size([32, 4096])
input_ids_1 torch.Size([32, 4096])
mask_1 torch.Size([32, 4096])
label torch.Size([32])


In [3]:
batch[1]['input_ids_0']

tensor([[99, 40, 50,  ...,  0,  0,  0],
        [99, 40, 50,  ...,  0,  0,  0],
        [99, 40, 50,  ...,  0,  0,  0],
        ...,
        [99, 40, 50,  ...,  0,  0,  0],
        [99, 40, 50,  ...,  0,  0,  0],
        [99, 40, 50,  ...,  0,  0,  0]])

In [4]:
from transformers import PerceiverForSequenceClassification
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
from transformers import PerceiverConfig
#get default perceiver config
configuration = PerceiverConfig()

#Print Updated Perceiver Configuration
print(configuration)

PerceiverConfig {
  "attention_probs_dropout_prob": 0.1,
  "audio_samples_per_frame": 1920,
  "cross_attention_shape_for_attention": "kv",
  "cross_attention_widening_factor": 1,
  "d_latents": 1280,
  "d_model": 768,
  "hidden_act": "gelu",
  "image_size": 56,
  "initializer_range": 0.02,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 2048,
  "model_type": "perceiver",
  "num_blocks": 1,
  "num_cross_attention_heads": 8,
  "num_frames": 16,
  "num_latents": 256,
  "num_self_attends_per_block": 26,
  "num_self_attention_heads": 8,
  "output_shape": [
    1,
    16,
    224,
    224
  ],
  "qk_channels": null,
  "samples_per_patch": 16,
  "self_attention_widening_factor": 1,
  "train_size": [
    368,
    496
  ],
  "transformers_version": "4.18.0",
  "use_query_residual": true,
  "v_channels": null,
  "vocab_size": 262
}



In [6]:
configuration.num_labels = 2
configuration.num_self_attends_per_block = 4
configuration.d_latents = 512
configuration.d_model = 128
configuration.max_position_embeddings = 4096
configuration.num_cross_attention_heads = 4
configuration.num_self_attention_heads = 4
configuration.num_latents = 128
configuration.vocab_size = 128

In [7]:
model = PerceiverForSequenceClassification(config = configuration)
model

PerceiverForSequenceClassification(
  (perceiver): PerceiverModel(
    (input_preprocessor): PerceiverTextPreprocessor(
      (embeddings): Embedding(128, 128)
      (position_embeddings): Embedding(4096, 128)
    )
    (embeddings): PerceiverEmbeddings()
    (encoder): PerceiverEncoder(
      (cross_attention): PerceiverLayer(
        (attention): PerceiverAttention(
          (self): PerceiverSelfAttention(
            (layernorm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
            (layernorm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
            (query): Linear(in_features=512, out_features=128, bias=True)
            (key): Linear(in_features=128, out_features=128, bias=True)
            (value): Linear(in_features=128, out_features=128, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): PerceiverSelfOutput(
            (dense): Linear(in_features=128, out_features=512, bias=True)
          )
        )
     

In [8]:
class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
        self.linear = nn.Linear(512,128)

    def forward(self, x):
        return self.linear(x)

In [9]:
model.perceiver.decoder.decoder.final_layer = Identity()

In [10]:
class Retrieval(nn.Module):
    def __init__(self,model):
        super(Retrieval, self).__init__()
        self.perceiver = model
        self.linear1 = nn.Linear(512,512)
        self.linear2 = nn.Linear(512,2)

    def forward(self,input_ids_0, mask_0, input_ids_1, mask_1):
        o1 = self.perceiver(inputs = input_ids_0, attention_mask = mask_0)
        o1 = o1.logits
        o2 = self.perceiver(inputs = input_ids_1, attention_mask = mask_1)
        o2 = o2.logits
        o3 = torch.cat([o1,o2,o1*o2,(o1 - o2)],axis = 1)
        o4 = F.relu(self.linear1(o3))
        o5 = self.linear2(o4)
        return o5

In [11]:
retriever = Retrieval(model).to(device)

In [12]:
retriever

Retrieval(
  (perceiver): PerceiverForSequenceClassification(
    (perceiver): PerceiverModel(
      (input_preprocessor): PerceiverTextPreprocessor(
        (embeddings): Embedding(128, 128)
        (position_embeddings): Embedding(4096, 128)
      )
      (embeddings): PerceiverEmbeddings()
      (encoder): PerceiverEncoder(
        (cross_attention): PerceiverLayer(
          (attention): PerceiverAttention(
            (self): PerceiverSelfAttention(
              (layernorm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
              (layernorm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
              (query): Linear(in_features=512, out_features=128, bias=True)
              (key): Linear(in_features=128, out_features=128, bias=True)
              (value): Linear(in_features=128, out_features=128, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): PerceiverSelfOutput(
              (dense): Linear(in_featur

In [13]:
pytorch_total_params = sum(p.numel() for p in retriever.parameters())
pytorch_total_params_Trainable = sum(p.numel() for p in retriever.parameters() if p.requires_grad)
print('Total Parameters: ', pytorch_total_params, '\nTrainable Parameters: ', pytorch_total_params_Trainable)  

Total Parameters:  9519362 
Trainable Parameters:  9519362


In [14]:
input_ids_0 = batch[1]["input_ids_0"].to(device)
mask_0 = batch[1]["mask_0"].to(device)
input_ids_1 = batch[1]["input_ids_0"].to(device)
mask_1 = batch[1]["mask_0"].to(device)
labels = batch[1]["label"].to(device)

In [15]:
outputs = retriever(input_ids_0,mask_0,input_ids_1,mask_1)

In [16]:
from torch.optim import AdamW
from tqdm.notebook import tqdm
from sklearn.metrics import accuracy_score
from datasets import load_metric
import pandas as pd
from torch.nn import CrossEntropyLoss

best_score = 0 
prev_score = 0
# maxPatience = 3
# currentPatience = 0

#steps = int(training_config["num_train_steps"]/20000)
steps = 20000
loss_fn = CrossEntropyLoss()

optimizer = AdamW(retriever.parameters(), 
                  lr = 0.01,
                  betas = (0.9, 0.999), 
                  eps = 1e-6, 
                  weight_decay = training_config["weight_decay"])

lr_scheduler = torch.optim.lr_scheduler.OneCycleLR(
    optimizer = optimizer,
    max_lr = training_config["learning_rate"],
    pct_start = training_config["warmup"] / training_config["num_train_steps"],
    anneal_strategy = training_config["lr_decay"],
    total_steps = training_config["num_train_steps"]
)

#amp_scaler = torch.cuda.amp.GradScaler() if model_config["mixed_precision"] else None

#initialize training summary
trainingSummary = pd.DataFrame(columns=['step', 'train_acc', 'val_acc'])


retriever.to(device)

#initialize training accuracy metric
train_accuracy = load_metric("accuracy")
#batch = next(ds_iter['train'])[1]

for step in tqdm(range(steps)):  # Perform gradient updates for multiple steps
    
    #model.train()
    retriever.train()
    
    #print("Step:", step)
    #for batch in tqdm(train_dataloader):
    batch = next(ds_iter['train'])[1]

    # get the inputs; 
    input_ids_0 = batch["input_ids_0"].to(device)
    mask_0 = batch["mask_0"].to(device)
    input_ids_1 = batch["input_ids_0"].to(device)
    mask_1 = batch["mask_0"].to(device)
    labels = batch["label"].to(device)

    # zero the parameter gradients
    optimizer.zero_grad()

    # forward + backward + optimize
    outputs = retriever(input_ids_0,mask_0,input_ids_1,mask_1)
    loss = loss_fn(outputs,labels)
    loss.backward()
    optimizer.step()
    lr_scheduler.step()

    # evaluate
    predictions = outputs.argmax(-1).cpu().detach().numpy()
    accuracy = accuracy_score(y_true=batch["label"].numpy(), y_pred=predictions)
    references = batch["label"].numpy()
    train_accuracy.add_batch(predictions=predictions, references=references)
    
    print(f"Loss: {loss.item()}, Accuracy: {accuracy}")

    #delete intermediate variables to free up GPU space
    del loss, outputs, input_ids_0, mask_0, labels, predictions, accuracy,input_ids_1,mask_1


    #Every 1000 steps validate and save model
    if (step+1)%training_config['eval_frequency']  == 0:
    #if (step+1)%2  == 0:
        #model.eval()
        retriever.eval()
        val_accuracy = load_metric("accuracy")

      #reset dev iterator
        ds_iter['dev'] = enumerate(DataLoader(val_dataset, batch_size = 32, drop_last = True))

        with torch.no_grad():
            for i, batch in tqdm(ds_iter['dev']):
                input_ids_0 = batch["input_ids_0"].to(device)
                mask_0 = batch["mask_0"].to(device)
                input_ids_1 = batch["input_ids_0"].to(device)
                mask_1 = batch["mask_0"].to(device)
                labels = batch["label"].to(device)

              # forward pass
                logits = retriever(input_ids_0,mask_0,input_ids_1,mask_1)
                predictions = logits.argmax(-1).cpu().detach().numpy()
                references = batch["label"].numpy()
                val_accuracy.add_batch(predictions=predictions, references=references)

          #delete intermediate variables to free up GPU space
                del logits, input_ids_0, mask_0, input_ids_1, mask_1, labels, predictions, references
      
      #Compute val accuracy
        final_val_score = val_accuracy.compute()['accuracy']
        print("Validation Accuracy:", final_val_score)

        if final_val_score >= best_score:
            best_score = final_val_score
            torch.save(retriever.to('cpu').state_dict(), piaynTaskModelDir + '/trainedPerceiverClassifierToken'+'.pkl')
            retriever.to(device)
        else:
            pass  

#         if final_val_score <= prev_score:
#             currentPatience += 1
#             if currentPatience >= maxPatience:
#                 print('Patience Limit reached! Stopping early!')
#                 torch.save(retriever.to('cpu').state_dict(), piaynTaskModelDir + '/trainedPerceiverClassifierStep_' + str(step + 1) + 'Token' + '.pkl')
#                 break  
#         else:
#             currentPatience = 0
      
      #Update prev_score
        prev_score = final_val_score

      #Compute training accuracy till now
        final_train_score = train_accuracy.compute()['accuracy']

      #Add to trainingSummary
        trainingSummary.loc[len(trainingSummary.index)] = [step+1, final_train_score, final_val_score]

  0%|          | 0/20000 [00:00<?, ?it/s]

Loss: 0.6964623928070068, Accuracy: 0.28125
Loss: 0.6920614838600159, Accuracy: 0.5625
Loss: 0.6880338788032532, Accuracy: 0.625
Loss: 0.6958847641944885, Accuracy: 0.46875
Loss: 0.7068854570388794, Accuracy: 0.34375
Loss: 0.6782148480415344, Accuracy: 0.6875
Loss: 0.7062689661979675, Accuracy: 0.375
Loss: 0.7035552859306335, Accuracy: 0.40625
Loss: 0.6943539977073669, Accuracy: 0.5
Loss: 0.6998217105865479, Accuracy: 0.4375
Loss: 0.702370285987854, Accuracy: 0.40625
Loss: 0.6941177248954773, Accuracy: 0.5
Loss: 0.6959140300750732, Accuracy: 0.46875
Loss: 0.6898190379142761, Accuracy: 0.5625
Loss: 0.6917625665664673, Accuracy: 0.53125
Loss: 0.6962832808494568, Accuracy: 0.4375
Loss: 0.6863601207733154, Accuracy: 0.6875
Loss: 0.6937285661697388, Accuracy: 0.5
Loss: 0.6942721009254456, Accuracy: 0.46875
Loss: 0.6900908946990967, Accuracy: 0.59375
Loss: 0.6898024678230286, Accuracy: 0.59375
Loss: 0.6926190853118896, Accuracy: 0.53125
Loss: 0.6947125792503357, Accuracy: 0.46875
Loss: 0.692

Loss: 0.6938621997833252, Accuracy: 0.28125
Loss: 0.6902278661727905, Accuracy: 0.59375
Loss: 0.6868714094161987, Accuracy: 0.59375
Loss: 0.6876381635665894, Accuracy: 0.5625
Loss: 0.7220043540000916, Accuracy: 0.34375
Loss: 0.6964555978775024, Accuracy: 0.5
Loss: 0.6918540000915527, Accuracy: 0.53125
Loss: 0.7219877243041992, Accuracy: 0.375
Loss: 0.7139380574226379, Accuracy: 0.40625
Loss: 0.6913580894470215, Accuracy: 0.53125
Loss: 0.6949721574783325, Accuracy: 0.5
Loss: 0.6769035458564758, Accuracy: 0.65625
Loss: 0.6975708603858948, Accuracy: 0.46875
Loss: 0.6913011074066162, Accuracy: 0.53125
Loss: 0.6966164708137512, Accuracy: 0.46875
Loss: 0.6934431195259094, Accuracy: 0.5
Loss: 0.6920357346534729, Accuracy: 0.53125
Loss: 0.6884680390357971, Accuracy: 0.625
Loss: 0.692032516002655, Accuracy: 0.53125
Loss: 0.696315586566925, Accuracy: 0.4375
Loss: 0.6909775733947754, Accuracy: 0.5625
Loss: 0.6979179978370667, Accuracy: 0.375
Loss: 0.688513457775116, Accuracy: 0.6875
Loss: 0.68820

0it [00:00, ?it/s]

Validation Accuracy: 0.5044800884955752
Loss: 0.6864565014839172, Accuracy: 0.5625
Loss: 0.6956619024276733, Accuracy: 0.5
Loss: 0.6866790056228638, Accuracy: 0.5625
Loss: 0.6957268118858337, Accuracy: 0.5
Loss: 0.6915709972381592, Accuracy: 0.53125
Loss: 0.6991113424301147, Accuracy: 0.46875
Loss: 0.705643892288208, Accuracy: 0.40625
Loss: 0.6884921193122864, Accuracy: 0.5625
Loss: 0.688983678817749, Accuracy: 0.5625
Loss: 0.6895068883895874, Accuracy: 0.5625
Loss: 0.6954882144927979, Accuracy: 0.46875
Loss: 0.6916778683662415, Accuracy: 0.53125
Loss: 0.6904792189598083, Accuracy: 0.5625
Loss: 0.6947800517082214, Accuracy: 0.46875
Loss: 0.6932843923568726, Accuracy: 0.5
Loss: 0.6966893672943115, Accuracy: 0.40625
Loss: 0.6955423951148987, Accuracy: 0.40625
Loss: 0.6936232447624207, Accuracy: 0.40625
Loss: 0.6937819123268127, Accuracy: 0.46875
Loss: 0.6954241991043091, Accuracy: 0.4375
Loss: 0.6933695077896118, Accuracy: 0.5
Loss: 0.6948210597038269, Accuracy: 0.46875
Loss: 0.699145376

...

Validation Accuracy: 0.5044800884955752
Loss: 0.6837450265884399, Accuracy: 0.75
Loss: 0.6945037245750427, Accuracy: 0.46875
Loss: 0.696953296661377, Accuracy: 0.40625
Loss: 0.6945955753326416, Accuracy: 0.46875
Loss: 0.6920017004013062, Accuracy: 0.53125
Loss: 0.6945587396621704, Accuracy: 0.46875
Loss: 0.6957370042800903, Accuracy: 0.4375
Loss: 0.6970227956771851, Accuracy: 0.40625
Loss: 0.6957088708877563, Accuracy: 0.4375
Loss: 0.692165195941925, Accuracy: 0.53125
Loss: 0.6988663077354431, Accuracy: 0.34375
Loss: 0.6888567805290222, Accuracy: 0.625
Loss: 0.6967398524284363, Accuracy: 0.40625
Loss: 0.6911716461181641, Accuracy: 0.5625
Loss: 0.6973560452461243, Accuracy: 0.375
Loss: 0.6930903792381287, Accuracy: 0.5
Loss: 0.6970851421356201, Accuracy: 0.375
Loss: 0.6941842436790466, Accuracy: 0.46875
Loss: 0.6905418634414673, Accuracy: 0.59375
Loss: 0.6871868371963501, Accuracy: 0.71875
Loss: 0.6915093064308167, Accuracy: 0.5625
Loss: 0.6914351582527161, Accuracy: 0.5625
Loss: 0.6949

Loss: 0.6933446526527405, Accuracy: 0.5
Loss: 0.6923748254776001, Accuracy: 0.53125
Loss: 0.6935118436813354, Accuracy: 0.5
Loss: 0.6948117017745972, Accuracy: 0.4375
Loss: 0.6950547695159912, Accuracy: 0.4375
Loss: 0.6917371153831482, Accuracy: 0.5625
Loss: 0.6941856145858765, Accuracy: 0.46875
Loss: 0.6908426284790039, Accuracy: 0.59375
Loss: 0.693210244178772, Accuracy: 0.5
Loss: 0.696885883808136, Accuracy: 0.375
Loss: 0.6925005912780762, Accuracy: 0.53125
Loss: 0.6915608048439026, Accuracy: 0.5625
Loss: 0.6941346526145935, Accuracy: 0.46875
Loss: 0.695006787776947, Accuracy: 0.4375
Loss: 0.6950223445892334, Accuracy: 0.4375
Loss: 0.6940974593162537, Accuracy: 0.46875
Loss: 0.6917480230331421, Accuracy: 0.5625
Loss: 0.693298876285553, Accuracy: 0.5
Loss: 0.6966075897216797, Accuracy: 0.34375
Loss: 0.693248987197876, Accuracy: 0.5
Loss: 0.6945005655288696, Accuracy: 0.4375
Loss: 0.69338059425354, Accuracy: 0.5
Loss: 0.6941471695899963, Accuracy: 0.4375
Loss: 0.6926814913749695, Accu

0it [00:00, ?it/s]

Validation Accuracy: 0.5044800884955752
Loss: 0.6932337284088135, Accuracy: 0.5
Loss: 0.6902064681053162, Accuracy: 0.59375
Loss: 0.6902060508728027, Accuracy: 0.59375
Loss: 0.6964052319526672, Accuracy: 0.40625
Loss: 0.6954377293586731, Accuracy: 0.4375
Loss: 0.6901078224182129, Accuracy: 0.59375
Loss: 0.6944115161895752, Accuracy: 0.46875
Loss: 0.6944280862808228, Accuracy: 0.46875
Loss: 0.6921364068984985, Accuracy: 0.53125
Loss: 0.6944461464881897, Accuracy: 0.46875
Loss: 0.6900790929794312, Accuracy: 0.59375
Loss: 0.6943082213401794, Accuracy: 0.46875
Loss: 0.6921787261962891, Accuracy: 0.53125
Loss: 0.6932071447372437, Accuracy: 0.5
Loss: 0.6885983943939209, Accuracy: 0.625
Loss: 0.6909654140472412, Accuracy: 0.5625
Loss: 0.6944323182106018, Accuracy: 0.46875
Loss: 0.6944425106048584, Accuracy: 0.46875
Loss: 0.6947031021118164, Accuracy: 0.46875
Loss: 0.6946461796760559, Accuracy: 0.46875
Loss: 0.6944944858551025, Accuracy: 0.46875
Loss: 0.6956416964530945, Accuracy: 0.4375
Loss:

Loss: 0.6951719522476196, Accuracy: 0.375
Loss: 0.6927692294120789, Accuracy: 0.53125
Loss: 0.6941229701042175, Accuracy: 0.4375
Loss: 0.6935958862304688, Accuracy: 0.46875
Loss: 0.6928175091743469, Accuracy: 0.53125
Loss: 0.6907262802124023, Accuracy: 0.6875
Loss: 0.6931877732276917, Accuracy: 0.5
Loss: 0.6919652819633484, Accuracy: 0.59375


In [17]:
from tqdm.notebook import tqdm
from datasets import load_metric

retriever.to(device)
retriever.eval()
test_accuracy = load_metric("accuracy")

with torch.no_grad():
    for i, batch in tqdm(ds_iter['test']):
        # get the inputs; 
        input_ids_0 = batch["input_ids_0"].to(device)
        mask_0 = batch["mask_0"].to(device)
        input_ids_1 = batch["input_ids_0"].to(device)
        mask_1 = batch["mask_0"].to(device)
        labels = batch["label"].to(device)

        # forward pass
        logits = retriever(input_ids_0,mask_0,input_ids_1,mask_1)
        predictions = logits.argmax(-1).cpu().detach().numpy()
        references = batch["label"].numpy()
        test_accuracy.add_batch(predictions=predictions, references=references)

          #delete intermediate variables to free up GPU space
        del logits, input_ids_0, mask_0, input_ids_1, mask_1, labels, predictions, references

final_score = test_accuracy.compute()
print("Accuracy on test set:", final_score['accuracy'])

0it [00:00, ?it/s]

Accuracy on test set: 0.5060503526982852
